## 1. Importing Libraries

In [1]:
!pip install keras tensorflow numpy

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\mgtoo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import nltk
import json
import pickle
import random
import numpy as np
from nltk.stem import WordNetLemmatizer
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
import os

-   `nltk` – Used for tokenization and lemmatization.
-   `json` – Loads the intent dataset from a JSON file.
-   `pickle` – Saves and loads processed data (`words.pkl`, `classes.pkl`).
-   `random` – Shuffles training data to improve generalization.
-   `numpy` – Handles data processing and numerical operations.
-   `WordNetLemmatizer` – Converts words to their base form (e.g., "running" → "run").
-   `keras.models.Sequential` – Defines the neural network architecture.
-   `keras.layers.Dense, Dropout` – Adds fully connected layers and dropout for regularization.
-   `keras.optimizers.SGD` – Uses Stochastic Gradient Descent for training.
-   `os` – Handles file operations.
---

## 2. Ensuring NLTK Resources are Available

In [3]:
# Function to check and download NLTK resources if not already available
def check_nltk_resources():
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt')
    
    try:
        nltk.data.find('corpora/wordnet')
    except LookupError:
        nltk.download('wordnet')

# Call the function to ensure NLTK resources are available
check_nltk_resources()

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


-   This function ensures that `punkt` (for tokenization) and `wordnet` (for lemmatization) are downloaded.
-   If they are missing, it downloads them.
---

## 3. Load and Process Data

In [4]:
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Load and process the data
data_file = open('Data/admission_data.json').read()
intents = json.loads(data_file)

-   `WordNetLemmatizer()` – Initializes the lemmatizer for text preprocessing.
-   `open('Data/admission_data.json').read()` – Opens and reads the JSON dataset.
-   `json.loads(data_file)` – Converts the JSON string into a Python dictionary.
---

## 4. Preparing Data for Training

In [5]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']

-   `words` – Stores unique words from all training sentences.
-   `classes` – Stores different intent tags.
-   `documents` – Stores word patterns mapped to intent tags.
-   `ignore_words` – Excludes punctuation marks.
---

In [6]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)  # Tokenize each sentence
        # Add to documents
        words.extend(w)  # Add words to list
        documents.append((w, intent['tag']))  # Store word-intent pair
        # Add to classes if not already present
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

-   Loops through each intent in the dataset.
-   Tokenizes each pattern sentence into words.
-   Stores word-tag pairs in `documents` for training.
-   Adds new intent tags to `classes`.
---

In [7]:
# Lemmatize and lower each word and remove duplicates
words = sorted(list(set([lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words])))

# Sort classes
classes = sorted(list(set(classes)))

-   Converts all words to lowercase and lemmatizes them.
-   Removes duplicates and sorts them.
-   Sorts `classes` to maintain consistency.
---

In [8]:
# Print data information
print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

91 documents
29 classes ['academic_support', 'admission_deadline', 'admission_interview', 'admission_requirements', 'admission_status', 'admission_tests', 'admission_website', 'application_fee', 'application_process', 'application_tips', 'campus_life', 'career_services', 'diversity', 'faculty', 'financial_aid', 'financial_aid_application', 'financial_planning', 'goodbye', 'greeting', 'health_services', 'housing', 'internships', 'online_resources', 'programs_offered', 'scholarship_deadline', 'student_activities', 'study_abroad', 'support_services', 'thanks']
145 unique lemmatized words ["'s", 'about', 'abroad', 'academic', 'access', 'act', 'activity', 'admission', 'advising', 'afford', 'aid', 'an', 'and', 'any', 'applicant', 'application', 'apply', 'are', 'attend', 'available', 'back', 'body', 'bye', 'campus', 'can', 'care', 'career', 'check', 'club', 'college', 'cost', 'counseling', 'course', 'day', 'deadline', 'decision', 'diversity', 'do', 'document', 'doe', 'dormitory', 'during', 'e

- Displays summary information about training data.
---

In [9]:
# Save words and classes to disk
pickle.dump(words, open('Model/words.pkl', 'wb'))
pickle.dump(classes, open('Model/classes.pkl', 'wb'))

- Saves processed `words` and `classes` for later use.
---

## 5. Creating Training Data

In [10]:
# Create training data
training = []
output_empty = [0] * len(classes)

-   `training` – Stores input-output training data.
-   `output_empty` – Initializes a list of zeros to represent class labels.
---

In [11]:
print(documents)

[(['Hi'], 'greeting'), (['Hello'], 'greeting'), (['Hey'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['What', 'is', 'the', 'application', 'deadline', '?'], 'admission_deadline'), (['When', 'do', 'I', 'need', 'to', 'apply', '?'], 'admission_deadline'), (['Deadline', 'for', 'application', '?'], 'admission_deadline'), (['What', 'are', 'the', 'admission', 'requirements', '?'], 'admission_requirements'), (['What', 'do', 'I', 'need', 'to', 'apply', '?'], 'admission_requirements'), (['What', 'documents', 'are', 'required', 'for', 'admission', '?'], 'admission_requirements'), (['What', 'programs', 'does', 'the', 'college', 'offer', '?'], 'programs_offered'), (['What', 'majors', 'are', 'available', '?'], 'programs_offered'), (['Lis

In [12]:
for doc in documents:
    # Initialize bag of words
    bag = []
    # Lemmatize each word
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in doc[0]]
    # Create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        # Fills `bag` with `1` if the word appears in the document, else `0`.
        
    # Create output array
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [13]:
print(bag)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]


- Creates an empty bag-of-words representation.
- Lemmatizes words in the current document.
- Fills `bag` with `1` if the word appears in the document, else `0`.
- Creates a one-hot encoded output array for the intent tag.
- Appends the `bag` (input) and `output_row` (label) to `training`.
---

In [14]:
# Shuffle the data and convert to numpy arrays
random.shuffle(training)
training = np.array(training, dtype=object)

-   Randomly shuffles data to prevent bias.
-   Converts `training` list into a NumPy array.

In [15]:
# Create train and test lists
train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

print("Training data created")

Training data created


- Splits `training` data into `train_x` (features) and `train_y` (labels).
- Confirms successful data preprocessing.

## 6. Creating and Training the Model

In [16]:
# Create model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

C:\Users\mgtoo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


-   `Sequential()` – Defines a feed-forward neural network.
-   `Dense(128, input_shape=(len(train_x[0]),), activation='relu')` – Adds a fully connected layer with 128 neurons and ReLU activation.
-   `Dropout(0.5)` – Prevents overfitting by randomly deactivating neurons.
-   `Dense(64, activation='relu')` – Adds another hidden layer with 64 neurons.
-   `Dense(len(train_y[0]), activation='softmax')` – Outputs probability distribution over intent classes.
---

In [17]:
# Compile model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

C:\Users\mgtoo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


-   Uses **Stochastic Gradient Descent (SGD)** as the optimizer.
-   `categorical_crossentropy` is the loss function (used for multi-class classification).

In [18]:
# Fit and save the model
hist = model.fit(train_x, train_y, epochs=1000, batch_size=5, verbose=1)

Epoch 1/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.0321 - loss: 3.3692  
Epoch 2/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0598 - loss: 3.3437
Epoch 3/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0722 - loss: 3.3014   
Epoch 4/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0587 - loss: 3.2531   
Epoch 5/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1360 - loss: 3.1932 
Epoch 6/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1337 - loss: 3.1280 
Epoch 7/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1023 - loss: 3.1760     
Epoch 8/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1325 - loss: 3.0775 
Epoch 9/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2498 - loss: 2.8818 
Epoch 10/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2200 - loss: 2.8788 
Epoch 11/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2049 - loss: 2.7472 
Epoch 12/1000
19/19 ━━━━━━━━━━━━━

-   Trains the model for **200 epochs** using mini-batches of size **5**.
-   `verbose=1` prints training progress.

In [19]:
model.save('Model/chatbot_model.h5', hist)
print("Model created")

Model created


- Saves the trained model as `chatbot_model.h5`.
- Confirms the successful creation of the model.
---
---

### **Summary of Key Steps**

1.  **Load and preprocess dataset** – Tokenization, lemmatization, and intent tagging.
2.  **Create bag-of-words representations** – Convert text data into numerical format.
3.  **Train a Neural Network** – Using dense layers and dropout to classify intent.
4.  **Save model and preprocessing files** – To be used in chatbot inference.

This is the complete breakdown of `train_model.py`. Let me know if you want to go deeper into any part! 🚀